## Confirmed Cases by Province/State (for Leaflet map)

In [1]:
import pandas as pd
import os
import json
import requests
from pprint import pprint

In [2]:
# Global Cases (minus U.S. total)
confirmed_df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
us = confirmed_df.loc[confirmed_df['Country/Region']=='US']
confirmed_df.drop(us.index[0], axis=0, inplace=True)
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,349,367,423,444,484,521,555,607,665,714
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,361,377,383,400,409,416,433,446,467,475
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1320,1423,1468,1572,1666,1761,1825,1914,1983,2070
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,501,525,545,564,583,601,601,638,646,659
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,14,16,17,19,19,19,19,19,19,19


In [3]:
# U.S. Cases by State
confirmed_us_df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_us_df = confirmed_us_df.iloc[:,6:]
confirmed_us_df.drop(['Combined_Key'], axis=1, inplace=True)
confirmed_us_df.rename(columns={'Province_State': 'Province/State',
                               'Country_Region': 'Country/Region',
                               'Long_': 'Long'}, inplace=True)
print(len(confirmed_us_df))
confirmed_us_df.head()

3256


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20
0,American Samoa,US,-14.2710,-170.1320,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,13.4443,144.7937,0,0,0,0,0,0,...,112,113,121,121,128,130,133,133,133,133
2,Northern Mariana Islands,US,15.0979,145.6739,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
3,Puerto Rico,US,18.2208,-66.5901,0,0,0,0,0,0,...,475,513,573,620,683,725,788,897,903,923
4,Virgin Islands,US,18.3358,-64.8963,0,0,0,0,0,0,...,42,43,43,45,45,50,51,51,51,51


In [4]:
# Drop rows with zero cases
last_column = confirmed_us_df[confirmed_us_df.columns[-1]]
rows = confirmed_us_df.loc[last_column == 0].index
confirmed_us_df.drop(rows, inplace=True)

# Drop cruise ship with no location
cruises = confirmed_us_df.loc[confirmed_us_df['Province/State'].str.contains('Grand Princess')].index
confirmed_us_df.drop(cruises, inplace=True)

print(len(confirmed_us_df))
confirmed_us_df.tail()

2700


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20
3248,Washington,US,0.0,0.0,0,0,0,0,0,0,...,533,648,677,915,838,908,848,732,619,484
3250,Wisconsin,US,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3253,Michigan,US,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,370,429
3254,Michigan,US,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,21,23
3255,Northern Mariana Islands,US,0.0,0.0,0,0,0,0,0,0,...,2,2,2,5,5,5,5,5,5,5


In [5]:
def lat_long(state):

    url = "https://google-maps-geocoding.p.rapidapi.com/geocode/json"

    querystring = {"language":"en","address":state}

    headers = {
        'x-rapidapi-host': "google-maps-geocoding.p.rapidapi.com",
        'x-rapidapi-key': GMAPS_API_KEY
        }

    response = requests.get(url, headers=headers, params=querystring).json()

    location = response['results'][0]['geometry']['location']
    lat = location['lat']
    long = location['lng']

    return lat, long

In [6]:
for index, row in confirmed_us_df.iterrows():
    state = row['Province/State']
    if ((row['Lat']==0) and (row['Long']==0) and (row['Country/Region']=='US')):
#         try:
        lat, long = lat_long(state)
        confirmed_us_df.loc[index, 'Lat'] = lat
        confirmed_us_df.loc[index, 'Long'] = long
#         except (IndexError, KeyError): 
#             print(index, state)
#             confirmed_us_df.drop(row, inplace=True)
#             pass

print(len(confirmed_us_df))

2700


In [7]:
confirmed_us_df

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20
1,Guam,US,13.444300,144.793700,0,0,0,0,0,0,...,112,113,121,121,128,130,133,133,133,133
2,Northern Mariana Islands,US,15.097900,145.673900,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6
3,Puerto Rico,US,18.220800,-66.590100,0,0,0,0,0,0,...,475,513,573,620,683,725,788,897,903,923
4,Virgin Islands,US,18.335800,-64.896300,0,0,0,0,0,0,...,42,43,43,45,45,50,51,51,51,51
5,Alabama,US,32.539527,-86.644082,0,0,0,0,0,0,...,12,12,12,12,15,17,19,19,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3248,Washington,US,47.751074,-120.740139,0,0,0,0,0,0,...,533,648,677,915,838,908,848,732,619,484
3250,Wisconsin,US,43.784440,-88.787868,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3253,Michigan,US,44.314844,-85.602364,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,370,429
3254,Michigan,US,44.314844,-85.602364,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,21,23


In [8]:
# Combine both Global and U.S.
combined_df = pd.concat([confirmed_df, confirmed_us_df], sort=False)
combined_df.drop(combined_df.columns[4:-1], axis=1, inplace=True)
combined_df.head()

,Province/State,Country/Region,Lat,Long,4/14/20
0,NaN,Afghanistan,33.0000,65.0000,714
1,NaN,Albania,41.1533,20.1683,475
2,NaN,Algeria,28.0339,1.6596,2070
3,NaN,Andorra,42.5063,1.5218,659
4,NaN,Angola,-11.2027,17.8739,19


In [10]:
# file = os.path.join('confirmed.csv')
# combined_df.to_csv(file, index=False, header=True)